In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow_recommenders as tfrs

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

Let's pretend we have 3 users:
1. Clarissa is a vegetarian who is lactose intolerant. 🥝🍅
2. Martin is an athlete who prefers meat dishes to vegetable dishes. 🍖🥩
3. Stella - has no special preferences, eats mostly everything. 🍲🍠

In [3]:
clarissa = {'id':'clr', 'prefers': ['vegetables', 'vegan', 'nomeat', 'vegetarian'], 'hates': ['meat', 'lactose']}
martin = {'id':'mrt', 'prefers': 'meat', 'hates': 'vegetables'}
stella = {'id':'stl', 'prefers': '', 'hates': ''}

We will additionally enter the factors Calories, Protein, Fat, Carbohydrates for our requirements. **(The numbers were taken at random and do not reflect proportions or recommendations - the numbers are just an example)**
* Clarissa - 2000/100/60/120
* Martin, 2600/150/100/328.
* Stella - 2200/80/50/100

In [4]:
def set_requirements(user, calories, proteins, fats, carbs):
  user['calories'] = calories
  user['proteins'] = proteins
  user['fats'] = fats
  user['carbs'] = carbs
set_requirements(clarissa, 2000, 100, 60, 120)
set_requirements(martin, 2600, 150, 100, 328)
set_requirements(martin, 2200, 80, 50, 100)


# Data processing 📊

Loading recipes and user_interactions. Nutritions showed like:calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV)

In [5]:
raw_recipes = pd.read_csv('./sample_data/RAW_recipes.csv')
raw_interactions = pd.read_csv('./sample_data/RAW_interactions.csv')
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


Merge data interactions with recipe infos

In [6]:
interactions_with_recipe_info = pd.merge(raw_interactions, raw_recipes[['name', 'tags', 'ingredients', 'id']], left_on='recipe_id', right_on='id', how='left')
interactions_with_recipe_info = interactions_with_recipe_info[['user_id', 'rating', 'name', 'tags']]
interactions_with_recipe_info.rename(columns={'name': 'recipe_name'}, inplace=True)
interactions_with_recipe_info.head()

,user_id,rating,recipe_name,tags
0,38094,4,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-..."
1,1293707,5,white bean green chile pepper soup,"['weeknight', 'time-to-make', 'course', 'main-..."
2,8937,4,devilicious cookie cake delights,"['30-minutes-or-less', 'time-to-make', 'course..."
3,126440,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course..."
4,57222,5,baked potato toppings,"['15-minutes-or-less', 'time-to-make', 'course..."


For liked user tags we need only high rated recipes

In [7]:
interactions_with_recipe_info.drop(interactions_with_recipe_info[interactions_with_recipe_info['rating'] < 4].index, inplace=True)

In [8]:
import ast

def convert_to_list(data_str):
  try:
    return ast.literal_eval(data_str)
  except (SyntaxError, ValueError):
    return []

In [9]:
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags'].apply(convert_to_list)
raw_recipes['tags_str'] = raw_recipes['tags'].apply(convert_to_list)

In [10]:
uniques_tags_array = np.array(open("./unique_tags.txt", 'r').read().split('\n'))

In [11]:
len(uniques_tags_array)

553

In [12]:
raw_recipes["tags_str"].max()

['weeknight',
 'time-to-make',
 'preparation',
 'very-low-carbs',
 'dietary',
 'low-saturated-fat',
 'low-calorie',
 'low-carb',
 'low-in-something',
 '4-hours-or-less']

In [13]:
tag_lookup_layer = tf.keras.layers.StringLookup(vocabulary=np.unique(uniques_tags_array), mask_token='empty', num_oov_indices=1)

In [14]:
tag_lookup_layer(interactions_with_recipe_info['tags_str'][:2][1])

<tf.Tensor: shape=(13,), dtype=int64, numpy=
array([541, 518, 135, 299, 402, 347, 472,  35, 533, 167, 141, 157, 178],
      dtype=int64)>

Formatting data for Datasets

In [15]:
interactions_with_recipe_info['user_id'] = interactions_with_recipe_info.user_id.astype("str")
interactions_with_recipe_info['rating'] = interactions_with_recipe_info.rating.astype(np.float32)
interactions_with_recipe_info['recipe_name'] = interactions_with_recipe_info.recipe_name.astype("str")
interactions_with_recipe_info['tags'] = interactions_with_recipe_info.tags.astype("str")

Creating train preferences for users

In [16]:
user_preferences = interactions_with_recipe_info[['user_id', 'rating', 'tags_str']]
user_preferences.head()

,user_id,rating,tags_str
0,38094,4.0,"[weeknight, time-to-make, course, main-ingredi..."
1,1293707,5.0,"[weeknight, time-to-make, course, main-ingredi..."
2,8937,4.0,"[30-minutes-or-less, time-to-make, course, mai..."
3,126440,5.0,"[15-minutes-or-less, time-to-make, course, mai..."
4,57222,5.0,"[15-minutes-or-less, time-to-make, course, mai..."


In [17]:
len(np.unique(user_preferences[user_preferences['user_id'] == '38094'].groupby('user_id')['tags_str'].sum()[0]))

C:\Users\Maincharter\AppData\Local\Temp\ipykernel_4496\1496501049.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  len(np.unique(user_preferences[user_preferences['user_id'] == '38094'].groupby('user_id')['tags_str'].sum()[0]))


151

In [18]:
user_liked_tags = user_preferences[user_preferences['rating'] >= 3].groupby('user_id')['tags_str'].apply(list).reset_index()
users_ds = pd.DataFrame(interactions_with_recipe_info['user_id'].unique(), columns=['user_id'])

In [19]:
users_ds = users_ds.merge(user_liked_tags, on='user_id', how='left')
users_ds = users_ds.rename(columns={'tags_str': 'liked_tags'})

In [20]:
users_ds.head()

,user_id,liked_tags
0,38094,"[[weeknight, time-to-make, course, main-ingred..."
1,1293707,"[[weeknight, time-to-make, course, main-ingred..."
2,8937,"[[30-minutes-or-less, time-to-make, course, ma..."
3,126440,"[[15-minutes-or-less, time-to-make, course, ma..."
4,57222,"[[15-minutes-or-less, time-to-make, course, ma..."


Looking that some of users don't have unliked_tags, so replace all Nan with empty arrays

In [21]:
def unique_tags_in_list(tags_list):
  if isinstance(tags_list, list):
    unique_tags = []
    for sublist in tags_list:
      if isinstance(sublist, list):
        for tag in sublist:
          if tag not in unique_tags:
            unique_tags.append(tag)
    return unique_tags
  else:
    return []

users_ds['liked_tags'] = users_ds['liked_tags'].apply(unique_tags_in_list)

In [22]:
users_ds['liked_tags'].max()

['weeknight',
 'time-to-make',
 'preparation',
 'very-low-carbs',
 'dietary',
 'low-saturated-fat',
 'low-calorie',
 'low-carb',
 'low-in-something',
 '4-hours-or-less',
 '60-minutes-or-less',
 'course',
 'cuisine',
 'occasion',
 'for-1-or-2',
 'main-dish',
 'easy',
 'european',
 'italian',
 'copycat',
 'low-sodium',
 'novelty',
 'number-of-servings']

In [24]:
users_ds.head()

,user_id,liked_tags
0,38094,"[weeknight, time-to-make, course, main-ingredi..."
1,1293707,"[weeknight, time-to-make, course, main-ingredi..."
2,8937,"[30-minutes-or-less, time-to-make, course, mai..."
3,126440,"[15-minutes-or-less, time-to-make, course, mai..."
4,57222,"[15-minutes-or-less, time-to-make, course, mai..."


In [25]:
interactions_with_recipe_info = pd.merge(interactions_with_recipe_info, users_ds, on='user_id', how='left')

Creating datasets

In [26]:
user_lengths = len(users_ds['liked_tags'].max())
recipe_lengths = len(interactions_with_recipe_info['tags_str'].max())
user_lengths, recipe_lengths

(23, 10)

In [27]:
# Параметры
user_lengths = len(users_ds['liked_tags'].max())
recipe_lengths = len(interactions_with_recipe_info['tags_str'].max())
MAX_TAG_LENGTH = max(user_lengths, recipe_lengths)
PADDING_VALUE = "empty"

interactions_with_recipe_info['liked_tags'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['liked_tags']
]

interactions_with_recipe_info['tags_str'] =  [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in interactions_with_recipe_info['tags_str'] ]

raw_recipes['tags_str'] = [
    (tags + [PADDING_VALUE] * (MAX_TAG_LENGTH - len(tags)))[:MAX_TAG_LENGTH] 
    for tags in raw_recipes['tags_str'] ]


In [28]:
interactions_with_recipe_info['liked_tags_str'] = interactions_with_recipe_info['liked_tags'].apply(lambda x: ','.join(x))
interactions_with_recipe_info['tags_str'] = interactions_with_recipe_info['tags_str'].apply(lambda x: ','.join(x))

In [29]:
def split_tags(tags_string):
    return tf.strings.split(tf.strings.strip(tags_string), ',')

In [30]:
tf.compat.v1.enable_eager_execution()

In [31]:
def convert_tags_to_indices(tags):
    # Применение слоя StringLookup для конверсии тегов в индексы
    return tag_lookup_layer(tags)

In [32]:
def squeeze_tags(data):
    return {
        "user_id": data["user_id"],
        "recipe_name": data["recipe_name"],
        "tags": tf.squeeze(data["tags"], axis=0),  # Укажите ось, которую нужно удалить
        "liked_tags": tf.squeeze(data["liked_tags"], axis=0),  # Укажите ось, которую нужно удалить
    }

# Шаг 2: Применение слоя в конвейере обработки данных
ratings = tf.data.Dataset.from_tensor_slices((
    tf.cast(interactions_with_recipe_info['user_id'].values, tf.string),
    tf.cast(interactions_with_recipe_info['recipe_name'].values, tf.string),
    tf.cast(interactions_with_recipe_info['tags_str'].values.reshape(-1,1), tf.string),
    tf.cast(interactions_with_recipe_info['liked_tags_str'].values.reshape(-1,1), tf.string),
)).map(lambda x, x1, x2, x3: {
    "user_id": x,
    "recipe_name": x1,
    "tags": convert_tags_to_indices(split_tags(x2)),  # Конверсия в индексы
    "liked_tags": convert_tags_to_indices(split_tags(x3)),  # Конверсия в индексы
}).map(squeeze_tags)

In [33]:
for data in ratings.take(1).as_numpy_iterator():
    print(data)

{'user_id': b'38094', 'recipe_name': b'white bean   green chile pepper soup', 'tags': array([541, 518, 135, 299, 402, 347, 472,  35, 533, 167, 141, 157, 178,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int64), 'liked_tags': array([541, 518, 135, 299, 402, 347, 472,  35, 533, 167, 141, 157, 178,
         9, 293, 199, 532, 534, 497,  56,   6, 287, 435], dtype=int64)}


In [34]:
raw_recipes['tags_str'] = raw_recipes['tags_str'].apply(lambda x: ','.join(x))

In [35]:
raw_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,tags_str
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,"60-minutes-or-less,time-to-make,course,main-in..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,"30-minutes-or-less,time-to-make,course,main-in..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"time-to-make,course,preparation,main-dish,chil..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,"60-minutes-or-less,time-to-make,course,main-in..."
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"weeknight,time-to-make,course,main-ingredient,..."


In [37]:
raw_recipes['id'] = raw_recipes.id.astype("str")
raw_recipes['name'] = raw_recipes.name.astype("str")
raw_recipes['tags'] = raw_recipes.tags.astype("str")
raw_recipes['ingredients'] = raw_recipes.ingredients.astype("str")

In [ ]:
raw_recipes['name'][:1]

In [39]:
# prompt: Создай на основе raw_recipes датасет, но учти что ingredients_str это массив

recipes = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['id'].values, tf.string),
                                              tf.cast(raw_recipes['name'].values, tf.string),
                                              tf.cast(raw_recipes['tags_str'].values.reshape(-1, 1), tf.string)
                                            #   tf.cast(raw_recipes['ingredients_str'].values.reshape(-1, 1), tf.string),
                                              )
                                             ).map(lambda x, x1, x2: {
                                                 'recipe_id': x,
                                                  "recipe_name": x1,
                                                  "tags": convert_tags_to_indices(split_tags(x2)),
                                                #   "ingredients": split_tags(x3)
                                              }).map(lambda data: {
        **data,
        "tags": tf.squeeze(data["tags"], axis=0)  # Удаляем дополнительные измерения
        # "ingredients": tf.squeeze(data["ingredients"], axis=0)  # Удаляем дополнительные измерения
    })



In [40]:
for data in recipes.take(1).as_numpy_iterator():
  print(data)

{'recipe_id': b'137739.0', 'recipe_name': b'arriba   baked winter squash mexican style', 'tags': array([  9, 518, 135, 299, 144, 402, 347, 337, 459, 533, 313, 167, 181,
       239, 534, 550, 157, 113, 450, 487,   0,   0,   0], dtype=int64)}


In [41]:
recipe_names = tf.data.Dataset.from_tensor_slices((tf.cast(raw_recipes['name'].values.reshape(-1, 1), tf.string))).map(lambda x: x)

# Towers 🗼

For our towers set dimensionality of the query and candidate representations: **32**. Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

In [179]:
#embedding_dimension = 32

## User tower 👷

Lets start creating our towers with User towers. We will compute by:
1. User id
2. Prefers and hates

### User ID model

In [42]:
unique_user_ids = users_ds["user_id"].unique()
unique_user_ids[:10]

array(['38094', '1293707', '8937', '126440', '57222', '52282', '76535',
       '273745', '353911', '190375'], dtype=object)

In [43]:
user_id_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids)+1, embedding_dimension)
])

NameError: name 'embedding_dimension' is not defined

### User Prefers models

## Recipe tower 🌭

In [44]:
unique_recipe_names = np.unique(list(raw_recipes["name"].unique()))

### Recipe ingredient&tag model

In [45]:
tags_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(uniques_tags_array) + 2, 160)  # Эмбеддинг без StringLookup
])

## Combine models

In [46]:
class UserModel(tf.keras.Model):
    def __init__(self, tags_model):
        super().__init__()
        

        
        # Liked tags embeddings
        self.liked_tags_model = tags_model

        # Penalty weight for unliked tags
        self.penalty_weight = -1.5
        self.plus_weight = 1.5

    def call(self, features):
        liked_tags = features["liked_tags"]

        # Embed user ID
        #user_id_embedding = tf.reduce_mean(user_id_embedding, axis=1)

        # Embed liked and unliked tags
        liked_tags_embedding = self.liked_tags_model(liked_tags)

        return liked_tags_embedding



In [47]:
# Извлечение данных пользователя
test_user = {}
for data in ratings.take(1).as_numpy_iterator():
    test_user = data
    
user_model = UserModel( tags_model)

user_model(test_user)

<tf.Tensor: shape=(23, 160), dtype=float32, numpy=
array([[ 0.01563456,  0.02409451,  0.00314189, ...,  0.01840777,
         0.03200716, -0.01909016],
       [-0.04498458,  0.01442958,  0.02059736, ..., -0.01981854,
         0.00275165,  0.04156375],
       [-0.02206186,  0.00399496, -0.01896117, ..., -0.00852623,
         0.03188828, -0.00303593],
       ...,
       [-0.00919078, -0.00968015, -0.04354283, ..., -0.00706878,
         0.04596254,  0.00838672],
       [ 0.00857923, -0.00993208, -0.01256803, ..., -0.04096646,
        -0.02391887,  0.04301068],
       [-0.0314896 ,  0.04797366, -0.01845132, ...,  0.01353276,
        -0.04841201, -0.04201857]], dtype=float32)>

In [48]:
class RecipeAndUserModel(tfrs.Model):

    def _reduce_mean_if_needed(self, embedding):
        """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
        if len(embedding.shape) >= 3:
            return tf.reduce_mean(embedding, axis=1)
        return embedding

    def __init__(self, unique_recipe_names, embedding_dimension):
        super().__init__()
        self.embedding_dimension = embedding_dimension
        # Модель пользователя
        
        self.tags_model = tf.keras.Sequential([
            tf.keras.layers.Embedding(len(uniques_tags_array) + 2, embedding_dimension)  # Эмбеддинг без StringLookup
        ])      

        self.user_model = UserModel(self.tags_model)

        # # Recipe embeddings
        # self.recipe_model = tf.keras.Sequential([
        #     tf.keras.layers.StringLookup(
        #         vocabulary=unique_recipe_names, mask_token=None),
        #     tf.keras.layers.Embedding(len(unique_recipe_names) + 1, self.embedding_dimension)
        # ])

        self.dense = tf.keras.layers.Dense(embedding_dimension)  # Сжимаем до размерности 32

        # Подготовка кандидатов, учитывающих и теги
        self.candidates = recipes.batch(128).map(self._get_candidate_embeddings)

        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.candidates
            )
        )
        
    def _get_candidate_embeddings(self, recipe_name):
        # Получаем эмбеддинги для имен и тегов
        tags_embedding = self.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
        tags_embedding = self._reduce_mean_if_needed(tags_embedding)
        # Объединяем эмбеддинги имен и тегов
        return tags_embedding

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # User embedding
        user_embedding = self.user_model(features)
        #recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # Tags embeddings
        tags = features["tags"]
        tags_embeddings = self.tags_model(tags)
        tags_embeddings = self._reduce_mean_if_needed(tags_embeddings)
        # # Вычисление средневзвешенного значения для тегов (можно модифицировать веса)
        # tags_embeddings_weighted_sum = tf.reduce_sum(tags_embeddings, axis=1)
        # sum_of_weights = tf.reduce_sum(tf.ones_like(tags_embeddings), axis=1)
        # tags_embeddings = tags_embeddings_weighted_sum / sum_of_weights
        
        # recipe_name_embeddings = self._reduce_mean_if_needed(recipe_name_embeddings)
        # tags_embeddings = tf.expand_dims(tags_embeddings, axis=0)
        # Конкатенация эмбеддингов рецепта и тегов
        

        return user_embedding, tags_embeddings

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings, recipe_embeddings = self(features)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        # Используем задачу факторизованного топ-K для оптимизации
        user_embeddings = self._reduce_mean_if_needed(user_embeddings)
        recipe_embeddings = self._reduce_mean_if_needed(recipe_embeddings)
        
        # user_embeddings = tf.keras.layers.Dense(64)(user_embeddings)
        # recipe_embeddings = tf.keras.layers.Dense(64)(recipe_embeddings)
        print("Computed: ", user_embeddings.shape, recipe_embeddings.shape)
        return self.task(user_embeddings, recipe_embeddings)


In [49]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(3_000)
test = shuffled.skip(80_000).take(3_000)

cached_train =  train.batch(4_000).cache()
cached_test = test.batch(15_000).cache()

In [50]:
model = RecipeAndUserModel(unique_recipe_names, 34)
early_callback = tf.keras.callbacks.EarlyStopping(monitor='loss',  patience=5, min_delta=0.001)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(cached_train, epochs=10)

Epoch 1/10
Computed:  (None, None, 34) (None, 34)
Computed:  (None, 34) (None, 34)
Computed:  (None, None, 34) (None, 34)
Computed:  (None, 34) (None, 34)
1/1 [==============================] - 25s 25s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0060 - factorized_top_k/top_50_categorical_accuracy: 0.0120 - factorized_top_k/top_100_categorical_accuracy: 0.0173 - loss: 24017.7734 - regularization_loss: 0.0000e+00 - total_loss: 24017.7734
Epoch 2/10
1/1 [==============================] - 15s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 24004.9902 - regularization_loss: 0.0000e+00 - total_loss: 24004.9902
Ep

In [51]:
model.evaluate(cached_test, return_dict=True)

Computed:  (None, None, 34) (None, 34)
Computed:  (None, 34) (None, 34)
1/1 [==============================] - 29s 29s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 3.3333e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0020 - factorized_top_k/top_100_categorical_accuracy: 0.0030 - loss: 23696.5469 - regularization_loss: 0.0000e+00 - total_loss: 23696.5469


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.00033333332976326346,
 'factorized_top_k/top_50_categorical_accuracy': 0.0020000000949949026,
 'factorized_top_k/top_100_categorical_accuracy': 0.003000000026077032,
 'loss': 23696.546875,
 'regularization_loss': 0,
 'total_loss': 23696.546875}

### Making predictions

In [ ]:
unique_tags[:20]

In [52]:
likes = np.full(MAX_TAG_LENGTH, "asian")
likes

array(['asian', 'asian', 'asian', 'asian', 'asian', 'asian', 'asian',
       'asian', 'asian', 'asian', 'asian', 'asian', 'asian', 'asian',
       'asian', 'asian', 'asian', 'asian', 'asian', 'asian', 'asian',
       'asian', 'asian'], dtype='<U5')

In [ ]:
dislikes = np.full(4, '30-minutes-or-less')
dislikes_2 = np.full(4, '60-minutes-or-less')
dislikes_3 = np.full(2, 'empty')
dislikes = np.append(dislikes, dislikes_2)
dislikes = np.append(dislikes, dislikes_3)
dislikes

In [ ]:
test_user['liked_tags']

In [ ]:
# Извлечение данных пользователя
test_user = {}
for data in test.take(1).as_numpy_iterator():
    test_user = data

# Создание набора данных для тестового пользователя
user_id_to_test = "42"  # Замените на нужный ID пользователя
test_user_dataset = {
    "user_id": np.array(['empty']),
    "liked_tags": np.array(dislikes),
    "unliked_tags": np.array(likes)
}

brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
def _reduce_mean_if_needed(embedding):
    """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
    if len(embedding.shape) >= 3:
        return tf.reduce_mean(embedding, axis=1)
    return embedding
dense = tf.keras.layers.Dense(365)

def _get_candidate_embeddings(recipe_name):
    # Получаем эмбеддинги для имен и тегов
    recipe_embedding =  model.recipe_model(recipe_name['recipe_name'])
    print(recipe_name['tags'])
    tags_embedding = model.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
    recipe_embedding = _reduce_mean_if_needed(recipe_embedding)
    tags_embedding = _reduce_mean_if_needed(tags_embedding)
    # Объединяем эмбеддинги имен и тегов
    combined_embedding = tf.concat([recipe_embedding, tags_embedding], axis=1)
    return combined_embedding
    
# Создание набора данных для рецептов и тегов
def get_recipe_and_tag_embeddings(x):
    recipe_embedding = model.recipe_model(x['recipe_name'])  # (None, 32)
    tag_embedding = model.tags_model(x['tags'])  # (None, 32)
    recipe_embedding =_reduce_mean_if_needed(recipe_embedding)
    tag_embedding = _reduce_mean_if_needed(tag_embedding)
    
    combined_embeddings = tf.concat([_reduce_mean_if_needed(recipe_embedding),  _reduce_mean_if_needed(tag_embedding)], axis=1)  # (None, 64)
    
    return dense(combined_embeddings)

# Используем map для обработки всех данных сразу
recipes_embeddings = recipes.batch(300).map(_get_candidate_embeddings)


# Создание набора имен рецептов
recipes_names = recipes.batch(300).map(lambda x: x["recipe_id"])  # Убедитесь, что это (None,)

# Объединяем эмбеддинги с именами
index = brute_force.index_from_dataset(
    tf.data.Dataset.zip((recipes_names, recipes_embeddings))
)

# Получаем рекомендации
_, titles = index(test_user_dataset, k=15)

# Вывод тегов для рекомендуемых рецептов
recommended_tags = raw_recipes[raw_recipes['id'].isin(titles.numpy().flatten().astype('str'))]


In [55]:
# Извлечение данных пользователя
test_user = {}
for data in test.take(1).as_numpy_iterator():
    test_user = data

# Создание набора данных для тестового пользователя
user_id_to_test = "42"  # Замените на нужный ID пользователя
test_user_dataset = {
    "liked_tags": tag_lookup_layer(likes),
}

brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
def _reduce_mean_if_needed(embedding):
    """Сокращает размерность эмбеддинга до среднего, если он многомерный."""
    if len(embedding.shape) >= 3:
        return tf.reduce_mean(embedding, axis=1)
    return embedding
dense = tf.keras.layers.Dense(365)

def _get_candidate_embeddings(recipe_name):
    tags_embedding = model.tags_model(recipe_name['tags'])  # Здесь предполагается, что recipe_name может содержать теги
    tags_embedding = _reduce_mean_if_needed(tags_embedding)
    return tags_embedding
    

# Используем map для обработки всех данных сразу
recipes_embeddings = recipes.batch(300).map(_get_candidate_embeddings)

# Печать для отладки
print("Embeddings:", recipes_embeddings)

# Создание набора имен рецептов
# Убедитесь, что здесь не выполняется попытка привести строку к float
recipes_names = recipes.batch(300).map(lambda x: x["recipe_id"])  # Убедитесь, что это (None,)


# Печать для отладки
print("Recipe names:", recipes_names)

# Объединяем эмбеддинги с именами
index = brute_force.index_from_dataset(
    tf.data.Dataset.zip((recipes_names, recipes_embeddings))
)

# Получаем рекомендации
_, titles = index(test_user_dataset, k=15)

# Вывод тегов для рекомендуемых рецептов
recommended_tags = raw_recipes[raw_recipes['id'].isin(titles.numpy().flatten().astype('str'))]


Embeddings: <_MapDataset element_spec=TensorSpec(shape=(None, 34), dtype=tf.float32, name=None)>
Recipe names: <_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>


In [ ]:
titles[0]

In [61]:
raw_recipes[raw_recipes['id'].isin(titles[0].numpy().flatten().astype('str'))][['tags_str','name']][:1]['tags_str']

1257    ,empty,empty,empty,empty,empty,empty,empty,emp...
Name: tags_str, dtype: object

In [ ]:
recommended_tags[['name', 'tags_str']]

In [ ]:
recommended_tags[['name', 'tags_str']]